In [1]:
import os
import csv
import json
import time
import tweepy
import datetime
import schedule

In [2]:
def initiate_api():
    with open('./.conf/config.json', 'r') as f:
        config = json.load(f)
    auth = tweepy.OAuthHandler(config["CONSUMER_KEY"], config["CONSUMER_SECRET"])
    auth.set_access_token(config["ACCESS_KEY"], config["ACCESS_SECRET"])
    api = tweepy.API(auth)
    return api

In [3]:
def get_woeid(api, locations):
    twitter_world = api.trends_available()
    places = {loc['name'].lower() : loc['woeid'] for loc in twitter_world};
    woeids = []
    for location in locations:
        if location in places:
            woeids.append(places[location])
        else:
            print("err: ",location," woeid does not exist in trending topics")
    return woeids

In [4]:
def isEnglish(text):
    try:
        text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [5]:
def get_tweets(api, query):
    
    tweets = []
    for status in tweepy.Cursor(api.search,
                                q=query,
                                count=100,
                                result_type='popular',
                                include_entities=True,
                                monitor_rate_limit=True, 
                                wait_on_rate_limit=True,
                                lang="en").items():
     
        if isEnglish(status.text) == True:
            tweets.append([status.id_str, 
                           query, 
                           status.created_at, 
                           status.user.screen_name, 
                           status.text])
    return tweets

In [6]:
def get_trending_hashtags(api, location):
    
    woeids = get_woeid(api, location)
    trending = set()
    
    for woeid in woeids:
        try:
            trends = api.trends_place(woeid)
        except:
            print("API limit exceeded. Waiting for next hour")
            time.sleep(3605) # 1-hour + 5 secs 
            trends = api.trends_place(woeid)
        
        topics = [trend['name'][1:] for trend in trends[0]['trends'] \
                  if (trend['name'].find('#') == 0 and isEnglish(trend['name']) == True)]
        
        trending.update(topics)
    
    return trending

In [7]:
def twitter_bot(api, locations):
    
    today = datetime.datetime.today().strftime('%Y_%m_%d')
    
    if not os.path.exists("trending_tweets"):
        os.makedirs("trending_tweets")
        
    file_tweets = open("trending_tweets/" + today + "-tweets.csv", "a+")
    file_hashtags = open("trending_tweets/" + today + "-hashtags.csv", "w+")
    writer = csv.writer(file_tweets)
    
    hashtags = get_trending_hashtags(api, locations)
    file_hashtags.write("\n".join(hashtags))
    print("Hashtags written to file.\n")
    file_hashtags.close()
    
    for hashtag in hashtags:
        try:
            print("Getting Tweets for the hashtag: ", hashtag)
            tweets = get_tweets(api, "#" + hashtag)
        except:
            print("API limit exceeded. Waiting for next hour")
            time.sleep(3605) # 1-hour + 5 secs
            tweets = get_tweets(api, "#" + hashtag)
        for tweet in tweets:
            writer.writerow(tweet)
    
    file_tweets.close()

In [8]:
import schedule
import time

def main():

    #locations = ['new york', 'los angeles', 'philadelphia', 'canada', 'united kingdom', 'india']        
    locations = ['new york']
    api = initiate_api()
    
    #schedule.every().day.at("00:00").do(twitter_bot, api, locations)
    schedule.every(905).seconds.do(twitter_bot, api, locations) # 15 mins + 5 secs
    
    while True:
        schedule.run_pending()
        time.sleep(5)

In [9]:
# run
main()

Hashtags written to file.

Getting Tweets for the hashtag:  GoAvsGo
Getting Tweets for the hashtag:  OneMission
Getting Tweets for the hashtag:  SexIsGreatButHaveYou
Getting Tweets for the hashtag:  OKCvsHOU
Getting Tweets for the hashtag:  miavsmil
Getting Tweets for the hashtag:  RHOBH
Getting Tweets for the hashtag:  AEWDynamite
Hashtags written to file.

Getting Tweets for the hashtag:  GoAvsGo
Getting Tweets for the hashtag:  OneMission
Getting Tweets for the hashtag:  NBAPlayoffs
Getting Tweets for the hashtag:  SexIsGreatButHaveYou
Getting Tweets for the hashtag:  miavsmil
Getting Tweets for the hashtag:  RHOBH
Getting Tweets for the hashtag:  AEWDynamite
Hashtags written to file.

Getting Tweets for the hashtag:  OneMission
Getting Tweets for the hashtag:  SexIsGreatButHaveYou
Getting Tweets for the hashtag:  OKCvsHOU
Getting Tweets for the hashtag:  RHOBH
Getting Tweets for the hashtag:  twug
Getting Tweets for the hashtag:  AEWDynamite
Hashtags written to file.

Getting Tweet

KeyboardInterrupt: 